In [28]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
#import seaborn as sns

In [11]:
#pull url
url = 'https://data.cdc.gov/resource/4ueh-89p9.json?$query=SELECT%0A%20%20%60data_as_of%60%2C%0A%20%20%60year%60%2C%0A%20%20%60education_level%60%2C%0A%20%20%60race_or_hispanic_origin%60%2C%0A%20%20%60sex%60%2C%0A%20%20%60age_group%60%2C%0A%20%20%60total_deaths%60%2C%0A%20%20%60covid_19_deaths%60'
r = requests.get(url)

# save data -> convert to df
data = r.json()
df = pd.DataFrame(data)
#df.head()




In [12]:
#clean data
#convert numeric values from strings
df['total_deaths'] = pd.to_numeric(df['total_deaths'])
df['covid_19_deaths'] = pd.to_numeric(df['covid_19_deaths'])
#strip age groups of extra text
df['age_group'] = df['age_group'].str.replace(" years", "")
df['race_or_hispanic_origin'] = df['race_or_hispanic_origin'].str.replace("Non-Hispanic ", "")
df['education_level'] = df['education_level'].str.replace("Bachelor�s degree or more", "Bachelor's degree or more")

#drop unneccesary frames.
df = df.drop("data_as_of", axis=1)
df = df.drop(df[df["education_level"] == "Unknown"].index)
df = df.drop(df[df["race_or_hispanic_origin"] == "Unknown"].index)
#df.head()

In [13]:
df["prop_dead"] = df.groupby(["education_level", "year", "race_or_hispanic_origin", "sex"])['total_deaths'].transform("sum")
df["prop_dead"] = df["total_deaths"] / df["prop_dead"]

df["prop_dead_covid"] = df.groupby(["education_level", "year", "race_or_hispanic_origin", "sex"])['covid_19_deaths'].transform("sum")
df["prop_dead_covid"] = df["covid_19_deaths"] / df["prop_dead_covid"]
#df.head(n = 20)

In [14]:
# print(df[df["year"]=="2019"].describe())

# print(df['education_level'].value_counts())
# print(df['sex'].value_counts())
# print(df['education_level'].value_counts())

# print(df['education_level'].unique())
# print(df['sex'].unique())
# print(df['race_or_hispanic_origin'].unique())
# print(df['year'].unique())


In [15]:
# df[df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["race_or_hispanic_origin","age_group"])["prop_dead"].mean()
# df[df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["education_level","age_group"])["prop_dead"].mean()

# df[df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["race_or_hispanic_origin","age_group"])["prop_dead_covid"].mean()
# df[df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["education_level","age_group"])["prop_dead_covid"].mean()


In [20]:
comb_df = df[df["year"] == "2020"]
other_df = df[df["year"] == "2019"]

comb_df = comb_df.merge(other_df[["education_level", "race_or_hispanic_origin", "sex", "age_group","prop_dead","total_deaths"]], 
                        on = ["education_level", "race_or_hispanic_origin", "sex", "age_group"],
                        suffixes=('_2020', '_2019'))
comb_df = comb_df.drop("year", axis = 1)
comb_df["prop_change"] = comb_df["prop_dead_covid"] - comb_df["prop_dead_2019"]
comb_df.head()

comb_df.to_csv("covid_death_demographics.csv", index = False)


In [19]:
print(comb_df['education_level'].unique())
print(comb_df['sex'].unique())
print(comb_df['race_or_hispanic_origin'].unique())

print(comb_df.describe())

# print(comb_df['education_level'].value_counts())
# print(comb_df['sex'].value_counts())
# print(comb_df['education_level'].value_counts())





['Associate degree or some college' "Bachelor's degree or more"
 'High school graduate/GED or less']
['Female' 'Male']
['Hispanic' 'American Indian or Alaska Native' 'Asian' 'Black'
 'More than one race' 'White']
       total_deaths_2020  covid_19_deaths  prop_dead_2020  prop_dead_covid  \
count         214.000000       214.000000      214.000000       214.000000   
mean        15459.341121      1743.070093        0.168219         0.168224   
std         50131.487827      5754.047053        0.147687         0.172858   
min             9.000000         0.000000        0.001032         0.000000   
25%           446.250000        33.500000        0.055666         0.022485   
50%          1710.500000       160.000000        0.139632         0.123526   
75%          9552.750000       907.000000        0.213361         0.253918   
max        530470.000000     59483.000000        0.731280         0.791629   

       prop_dead_2019  total_deaths_2019   prop_change  
count      214.000000      

In [26]:
comb_df[comb_df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["age_group","race_or_hispanic_origin"])["prop_change"].mean()
comb_df[comb_df["age_group"].isin(["0-24","25-39","40-54"])].groupby(["age_group","education_level"])["prop_change"].mean()

age_group  education_level                 
0-24       Associate degree or some college   -0.020263
           Bachelor's degree or more          -0.004370
           High school graduate/GED or less   -0.063852
25-39      Associate degree or some college   -0.044781
           Bachelor's degree or more          -0.025146
           High school graduate/GED or less   -0.035442
40-54      Associate degree or some college   -0.015165
           Bachelor's degree or more          -0.019309
           High school graduate/GED or less   -0.019589
Name: prop_change, dtype: float64